Case técnico - Analista de dados III - Igor Ralf da Silva

Seção 1 - SQL

    Para iniciar a resolução do problema, é necessário entender como os dados estão dispostos dentro das tabelas e quais os possíveis problemas dentro da sua estrutura. Para garantir a solução adequada são necessárias algumas premissas:

1. A tb_cpv_historico pode conter ou não dados de 2023, ou seja, para as tabelas **_tb_cpv_ecomm_linx_** e **_tb_cpv_ecomm_** serão usados apenas os dados de 2023
2. Como entre as tabelas _**tb_cpv_ecomm_linx**_ e **_tb_cpv_ecomm_** não existem sobreposições, posso unir todos os dados das duas tabelas em apenas uma
3. Dentro da tabela **_tb_cpv_ecomm_linx_** é necessário construir a formatação correta com o "0" antes do dado
4. O CPV do combo é determinado pelo somatório do CPV de seus itens, logo, não posso extrair o valor final direto pela tabela **_tb_produto_skus_** por não saber o ‘input’ inicial desses dados, por tanto, os combos serão removidos dela
5. Como COD_MATERIAL_FILHO é o código de cada produto do combo, logo, o somatório dos CPVs do COD_MATERIAL_FILHO será o CPV do COD_MATERIAL_PAI
6. O formato dos campos é desconhecido (‘string’, integer, float), portanto, mesmo que a grafia seja, por exemplo, de booleano, posso estar a trabalhar com um campo em ‘string’

Para resolução do primeiro ponto ao terceiro ponto serão construídas as tabelas filtradas para 2023 e com os campos necessários alterados

In [ ]:
%%sql
SELECT COD_UN_NEGOCIO, CAST(COD_MATERIAL AS STRING) AS COD_MATERIAL, ANO, MIN(CPV)
      FROM tb_cpv_ecomm
      where CAST(ANO AS STRING) = "2023"
      GROUP BY COD_MATERIAL, COD_UN_NEGOCIO, ANO, FLG_COMBO
      UNION ALL
      (SELECT
        COD_UN_NEGOCIO,
        CASE WHEN STARTS_WITH(CAST(COD_MATERIAL AS STRING), '0')
        THEN CAST(COD_MATERIAL AS STRING) ELSE CONCAT('0', CAST(COD_MATERIAL AS STRING))
        END AS COD_MATERIAL,
        ANO, MIN(CPV)
      FROM tb_cpv_ecomm_linx AS B
      where CAST(ANO AS STRING) = "2023"
      GROUP BY COD_MATERIAL, COD_UN_NEGOCIO, ANO, FLG_COMBO)


Em seguida, é necessário construir a estrutura da tabela de **_tb_produto_skus_** de maneira filtrada para remover os combos e evitar um processamento excessivo

In [ ]:
%%sql
  SELECT CAST(COD_MATERIAL AS STRING) AS COD_MATERIAL, COD_UN_NEGOCIO
  FROM tb_produto_skus
  where LOWER(CAST (flg_combo as STRING)) = ('false')

Agora preciso criar uma tabela de histórico pegando apenas o minimo do cpv

In [ ]:
%%sql
    SELECT COD_UN_NEGOCIO, CAST(COD_MATERIAL AS STRING) AS COD_MATERIAL, ANO, MIN(CPV)
    FROM tb_cpv_historico
    GROUP BY COD_MATERIAL, COD_UN_NEGOCIO, ANO, FLG_COMBO

Compilando os valores de CPV

In [ ]:
%%sql
SELECT COALESCE(A.COD_UN_NEGOCIO, B.COD_UN_NEGOCIO) AS COD_UN_NEGOCIO,
       COALESCE(A.COD_MATERIAL, B.COD_MATERIAL) AS COD_MATERIAL,
       COALESCE(A.ANO, B.ANO) AS ANO,
       LEAST(A.CPV, B.CPV) AS CPV
    FROM tb_cpv_historico_ajust AS A
    FULL JOIN tb_cpv_linx_ecomm as B
    ON A.COD_UN_NEGOCIO = B.COD_UN_NEGOCIO
    AND A.ANO = B.ANO
    AND A.COD_MATERIAL = B.COD_MATERIAL


Agora preciso agregar os valores de CPV para os produtos que são combo e para os que não são combo

In [ ]:
%%sql
--produtos_nao_combo
SELECT A.COD_MATERIAL, A.COD_UN_NEGOCIO, B.CPV, 0 as FLG_COMBO, B.ANO
FROM tb_produto_skus_filtrada as A


INNER JOIN
    CPV_COMP  AS B
ON A.COD_UN_NEGOCIO = B.COD_UN_NEGOCIO
AND A.COD_MATERIAL = B.COD_MATERIAL



In [ ]:
%%sql
--produtos_combo as

  SELECT A.COD_MATERIAL_PAI AS COD_MATERIAL, A.COD_UN_NEGOCIO, B.ANO,
  SUM (B.CPV) AS CPV, 1 AS FLG_COMBO
FROM tb_produto_sku_combo AS A
INNER JOIN
CPV_COMP as B
ON A.COD_UN_NEGOCIO = B.COD_UN_NEGOCIO
AND A.COD_MATERIAL_FILHO = B.COD_MATERIAL
GROUP BY A.COD_MATERIAL_PAI, B.ANO, A.COD_UN_NEGOCIO



Agregando todo o código e usando uma estrutura de CTEs, obtemos o código a seguir:

In [ ]:
%%sql
WITH
tb_cpv_linx_ecomm as
    (
      SELECT COD_UN_NEGOCIO, CAST(COD_MATERIAL AS STRING) AS COD_MATERIAL, ANO, MIN(CPV)
      FROM tb_cpv_ecomm
      where CAST(ANO AS STRING) = "2023"
      GROUP BY COD_MATERIAL, COD_UN_NEGOCIO, ANO, FLG_COMBO
      UNION ALL
      (SELECT
        COD_UN_NEGOCIO,
        CASE WHEN STARTS_WITH(CAST(COD_MATERIAL AS STRING), '0')
        THEN CAST(COD_MATERIAL AS STRING) ELSE CONCAT('0', CAST(COD_MATERIAL AS STRING))
        END AS COD_MATERIAL,
        ANO, MIN(CPV)
      FROM tb_cpv_ecomm_linx AS B
      where CAST(ANO AS STRING) = "2023"
      GROUP BY COD_MATERIAL, COD_UN_NEGOCIO, ANO, FLG_COMBO)
    ),


tb_produto_skus_filtrada AS
  (
      SELECT CAST(COD_MATERIAL AS STRING) AS COD_MATERIAL, COD_UN_NEGOCIO
      FROM tb_produto_skus
      where LOWER(CAST (flg_combo as STRING)) = ('false')
  ),


tb_cpv_historico_ajust AS
    (
        SELECT COD_UN_NEGOCIO, CAST(COD_MATERIAL AS STRING) AS COD_MATERIAL, ANO, MIN(CPV)
        FROM tb_cpv_historico
        GROUP BY COD_MATERIAL, COD_UN_NEGOCIO, ANO, FLG_COMBO
    ),


CPV_COMP AS
    (
        SELECT COALESCE(A.COD_UN_NEGOCIO, B.COD_UN_NEGOCIO) AS COD_UN_NEGOCIO,
        COALESCE(A.COD_MATERIAL, B.COD_MATERIAL) AS COD_MATERIAL,
        COALESCE(A.ANO, B.ANO) AS ANO,
        LEAST(A.CPV, B.CPV) AS CPV
        FROM tb_cpv_historico_ajust AS A
        FULL JOIN tb_cpv_linx_ecomm as B
        ON A.COD_UN_NEGOCIO = B.COD_UN_NEGOCIO
        AND A.ANO = B.ANO
        AND A.COD_MATERIAL = B.COD_MATERIAL

    ),




produtos_nao_combo as

    (
        SELECT A.COD_MATERIAL, A.COD_UN_NEGOCIO, B.CPV, 0 as FLG_COMBO, B.ANO
        FROM tb_produto_skus_filtrada as A
        INNER JOIN
            CPV_COMP  AS B
        ON A.COD_UN_NEGOCIO = B.COD_UN_NEGOCIO
        AND A.COD_MATERIAL = B.COD_MATERIAL
    ),

produtos_combo as

    (
        SELECT A.COD_MATERIAL_PAI AS COD_MATERIAL, A.COD_UN_NEGOCIO, B.ANO,
        SUM (B.CPV) AS CPV, 1 AS FLG_COMBO
        FROM tb_produto_sku_combo AS A
        INNER JOIN
        CPV_COMP as B
        ON A.COD_UN_NEGOCIO = B.COD_UN_NEGOCIO
        AND A.COD_MATERIAL_FILHO = B.COD_MATERIAL
        GROUP BY A.COD_MATERIAL_PAI, B.ANO, A.COD_UN_NEGOCIO
    )


SELECT
  COD_MATERIAL,
  COD_UN_NEGOCIO,
  CPV,
  ANO,
  FLG_COMBO
FROM
  produtos_nao_combo
UNION ALL
SELECT
  COD_MATERIAL,
  COD_UN_NEGOCIO,
  CPV,
  ANO,
  FLG_COMBO
FROM
  produtos_combo


Para a validação da tabela, além da satisfação das premissas antes adotadas seria necessário verificar se os valores de cpv estão devidamente preenchidos, se os códigos material estão na formatação correta, algumas validações como formato de exibição do ano seriam necessárias antes da implementação do modelo, e os formatos dos campos, a query apresentada minimiza o erro em alguns casos, como o campo de cod_material em _‘string’_, mas não para todos os campos, além disso, é necessário verificar o preenchimento do campo unidade de negócio, foi inserido um campo de combo para identificação apenas, mas seria importante para a tabela final a verificação de duplicidade.

Seção 2 - Python + Análise de dados


    OBS.: O compilador usado foi o pycharm com uma alocação máxima de 8gb de memória, o que limita o uso de modelos mais rebuscados

    Para iniciar a construção do modelo é necessário estudar a qualificação dos modelos, para isso, é necessário compreender os dados disponíveis:

1. É necessário um modelo que gere 3 clusteres finais
2. Não há uma pré-classificação dos dados
3. Os comentários podem conter erros de grafia, problemas de acentuação e outros elementos não textuais em seu corpo
4. Não serão feitas alterações na tabela de comentários inicial manualmente
5. O modelo será construído como uma versão inicial e serão sugeridas ao final melhorias a ele

Com base nas premissas adotadas, inicialmente, foram agregados os dados de review e vendas em um único arquivo para unificação da fonte de dados. Em seguida, foi iniciada a construção lógica do modelo:

    O modelo não possui pré-classificação então é um modelo não supervisionado, com 3 clusters finais, usando uma linguagem natural em língua portuguesa, com esses fatos, existem 2 opções que foram testadas:

1. Usar um modelo do zero, ou seja, construir as etapas de preprocessamento, em que seria necessário transformar as letras para minúsculas, selecionar os caracteres coerentes com a língua portuguesa e vetorizar esses dados. Um erro encontrado era de palavras sem significado específico, como conjunções, então foi usada a biblioteca _nltk_ para remover _stopwords_ incoerentes, um dicionário (_floresta_) para garantir a presença apenas de palavras existentes e o _spacy_ para seleção apenas de adjetivos. Contudo, a conversão não foi como esperado e esse modelo **foi descontinuado**.
2. Esse segundo modelo foi o aplicado ao final, ele utiliza uma biblioteca chamada **BERT** que é uma rede neural de palavras que é sensível a coloquialismos e ironias na classificação, a própria biblioteca possui tratamento para stopwords, palavras incorretas, esse modelo faria a construção dos vetores.


    Contudo, o BERT por si só não classifica o texto, então foram avaliados diversos modelos, muitos deles sendo desclassificados por conta da nuance de 3 clusteres, inclusive os modelos mais usados com o BERT e BERTopic, o DBScan e o HDBScan, respectivamente. Foram testados os modelos de GMM, clusterização hierárquica, K-medoids, K-Median e K-Means. Sendo o K-Means o modelo escolhido.

1. O K-Means é um modelo simples e bastante utilizado, não possui boa gestão de outliers, mas possui baixo custo de processamento
2. Os modelos de GMM e K-Median não conseguiram convergir com o Slot de RAM alocado
3. A clusterização hierárquica leva muito tempo de execução, o que pode não ser eficiente em dados que se alterem bastante ou bigdata
4. O modelo de K-medoids apresentou uma composição de palavras mais dispersa que o K-means dificultando os insights


Por fim, as bibliotecas usadas foram:

1. ntlk
2. pandas
3. numpy
4. skcit-learn
5. umap-learn
6. sentence-transformers (bert)
7. matplotlib
8. seaborn
9. nltk

E para iniciar o código, vamos instalá-las:

In [ ]:
pip install pandas
pip install numpy
pip install scikit-learn
pip install umap-learn
pip install sentence-transformers
pip install matplotlib
pip install seaborn
pip install nltk

Após a instalação no terminal das bibliotecas, o código será o seguinte:

In [ ]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import silhouette_score
from umap import UMAP
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns

#Baixar as stopwords
nltk.download('stopwords')

stop_words_pt = set(stopwords.words('portuguese'))
stop_words_pt.discard('não')
stop_words_pt.discard('mais')
stop_words_pt.discard('muito')

'''logica de importação e conversão em dataframe,
a coluna de cod_material equivale ao valor da coluna de cod_produto e será renomeada para melhor coerência no join'''

df_reviews = pd.read_csv('reviews.csv')
df_vendas = pd.read_csv('vendas.csv')
df_vendas_ajuste = df_vendas.rename(columns={'COD_MATERIAL': 'COD_PRODUTO'})
df_comp = pd.merge(df_reviews, df_vendas_ajuste, on=['COD_PEDIDO', 'COD_PRODUTO'], how='left')
df = pd.DataFrame(df_comp)

#criar o campo de ano com mês e remover as outras informações
df['DT_HR_CRIACAO'] = pd.to_datetime(df['DT_HR_CRIACAO'], errors='coerce')
df['ANO_MES'] = df['DT_HR_CRIACAO'].dt.to_period('M')

#Vou agrupar por cod_produto e por mês, então preciso remover do dataframe os valores nulos para cod_produto
df.dropna(subset=['COD_PRODUTO', 'DT_HR_CRIACAO', 'MSG_AVALIACAO'], inplace=True)
df['COD_PRODUTO'] = df['COD_PRODUTO'].astype(int)

df['cluster_kmeans'] = np.nan #definir o valor inicial do "cluster" como uma coluna sem valores

groups = df.groupby(['COD_PRODUTO', 'ANO_MES'])

embedding_model_name = "neuralmind/bert-base-portuguese-cased"
bert_model = SentenceTransformer(embedding_model_name)

#foram testados diversos neighbors e n_components para melhor conversão

umap_model = UMAP(n_neighbors=15, n_components=3, min_dist=0.0, metric='cosine', random_state=42)

kmeans_model = KMeans(n_clusters=3, n_init=50, random_state=42) #poucas inicializações para um modelo inicial

lista_silhuetas=[]
lista_contagem_clusters=[]

for (COD_PRODUTO, ANO_MES), df_group in groups:

    textos_para_analise = [str(t) for t in df_group['MSG_AVALIACAO'].tolist()]

    document_embeddings = bert_model.encode(textos_para_analise)

    reduced_embeddings = umap_model.fit_transform(document_embeddings)

    kmeans_model.fit(reduced_embeddings)
    clusters = kmeans_model.predict(reduced_embeddings)

    print(f"Produto e mês: {COD_PRODUTO} e {ANO_MES}")

    contagem_clusters = pd.Series(clusters).value_counts().to_dict()
    print(f"\nPopulação dos clusters: {contagem_clusters}")


    lista_contagem_clusters.append({'COD_PRODUTO': COD_PRODUTO, 'ANO_MES': ANO_MES,
                                    'contagem_clusters': contagem_clusters
    })


    silhueta = silhouette_score(reduced_embeddings, clusters)
    print(f"Coeficiente de Silhueta: {silhueta:.2f}\n")
    lista_silhuetas.append({'COD_PRODUTO': COD_PRODUTO, 'ANO_MES': str(ANO_MES),
                            'Silhueta': silhueta
                            })

#Aqui o código, propriamente dito, já foi executado e daqui pra frente existirão regras de export e visualização

    df.loc[df_group.index, 'cluster_kmeans'] = clusters

    vectorizer = TfidfVectorizer(encoding="utf-8", max_features=30, stop_words=list(stop_words_pt), ngram_range=(2, 3))
    x_tfidf = vectorizer.fit_transform(df_group['MSG_AVALIACAO'])

#foram vetorizadas as palavras para conseguirmos visualizá-las matricialmente, o tfidvectorizer foi algo usado no modelo descontinuado

    top_words_data = []

    for c in np.unique(clusters):
        indice = np.where(clusters == c)[0]
        tfidf = x_tfidf[indice]

        mean_tfidf = np.array(tfidf.mean(axis=0))

        top_indices = mean_tfidf.argsort()[-10:][::-1]
        top_words = [vectorizer.get_feature_names_out()[i] for i in top_indices]

        top_words_data.append({'cluster': c, 'top_words': top_words})
        top_words_list = [str(word) for word in top_words]
        print(f"  Cluster {c}: {', '.join(top_words_list)}")

    df_top_words = pd.DataFrame(top_words_data)
    df_top_words.to_csv(f'palavras_chave_p{COD_PRODUTO}_m{ANO_MES}.csv', index=False)

    if 'ESTADO_AVALIADOR' in df_group.columns and not df_group['ESTADO_AVALIADOR'].isnull().all():
        plt.figure(figsize=(12, 6))
        sns.countplot(data=df, x='ESTADO_AVALIADOR', hue='cluster_kmeans')
        plt.title(f'Distribuição de Clusters por Estado para o Produto {COD_PRODUTO} ({ANO_MES})')
        plt.xlabel('Estado do Avaliador')
        plt.ylabel('Contagem de Avaliações')
        plt.legend(title='Cluster')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'clusters_estado_p{COD_PRODUTO}_m{ANO_MES}.png')
        plt.close()

    if 'DES_CANAL_VENDA_FINAL' in df_group.columns and not df_group['DES_CANAL_VENDA_FINAL'].isnull().all():
        plt.figure(figsize=(12, 6))
        sns.countplot(data=df, x='DES_CANAL_VENDA_FINAL', hue='cluster_kmeans')
        plt.title(f'Distribuição de Clusters por Canal de Venda para o Produto {COD_PRODUTO} ({ANO_MES})')
        plt.xlabel('Canal de Venda')
        plt.ylabel('Contagem de Avaliações')
        plt.legend(title='Cluster')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig(f'clusters_canal_venda_p{COD_PRODUTO}_m{ANO_MES}.png')
        plt.close()

if lista_silhuetas:
    df_silhuetas = pd.DataFrame(lista_silhuetas)
    df_silhuetas.to_csv('silhueta_kmeans_bert_mes_produto.csv', index=False)

if lista_contagem_clusters:
    df_contagem = pd.DataFrame(lista_contagem_clusters)
    df_contagem.to_csv('contagem_kmeans_bert_mes_produto.csv', index=False)

df_final = df[['COD_PRODUTO', 'ANO_MES', 'MSG_AVALIACAO', 'cluster_kmeans', 'ESTADO_AVALIADOR', 'DES_CANAL_VENDA_FINAL', 'FLG_PRESENTE']]
df_final.to_csv('compiladofinal_kmeans_bert_mes_produto.csv', index=False)

#Específico para presentes

if 'FLG_PRESENTE' in df.columns and not df['FLG_PRESENTE'].isnull().all():
    for prod_id in df['COD_PRODUTO'].unique():
        df_prod = df[df['COD_PRODUTO'] == prod_id]

        if not df_prod.empty:
            plt.figure(figsize=(12, 6))
            sns.countplot(data=df_prod, x='cluster_kmeans', hue='FLG_PRESENTE')
            plt.title(f'População de Clusters e Distribuição de Presente para o Produto {prod_id}')
            plt.xlabel('Cluster')
            plt.ylabel('Contagem de Avaliações')
            plt.legend(title='Presente (FLG_PRESENTE)')
            plt.xticks(rotation=0)
            plt.tight_layout()
            plt.savefig(f'clusters_flg_presente_p{prod_id}.png')
            plt.close()

df_produto_333 = df_final[df_final['COD_PRODUTO'] == 333].copy()
df_produto_333 = df_produto_333.sort_values(by='cluster_kmeans')
df_produto_333.to_csv('compilado_produto_333.csv', index=False)

Sugestões ao modelo:

1. Após a execução do código, atribuir os parâmetros negativo, positivo e neutro ao modelo e, reinserir os dados num modelo de aprendizagem supervisionada
2. Pré-categorizar um grupo de dados para aumentar a assertividade e substituir o método por um modelo supervisionado
3. Determinar um

Perguntas:
1) Sabendo que o produto 333 foi lançado em 02/2025, como pode ser explicada a
mudança de comportamento no sentimento das avaliações entre os meses 02, 03 e
04?

    R. No geral, para os 3 meses, o produto possui diversos comentários sobre a eficácia, nos dois primeiros meses a silhueta foi menor e todos os ‘clusters’ se assemelhavam mais, sendo que o principal argumento era sobre a baixa remoção de pelos. Após esse mês, surgiram palavras que apontavam para casos em que funcionava e em que não, com alguns comentários bem positivos e outros negativos, no terceiro e quarto meses as palavras sobre o cheiro aumentaram e começaram a se formar opiniões divididas. Basicamente, no primeiro mês foram coletadas mais opiniões e a eficácia do produto foi bem questionada, em seguida, algumas pessoas já estavam cientes do uso do produto e começaram a ter resultados, enquanto outras não, ao final, as opiniões dividiram, mas pontos como cheiro e eficácia continuam em alta, exigindo um olhar mais cuidadoso

2) Entre os produtos 111 e 222, algum deles é mais indicado para uma campanha de
incentivo a compras para presentear? Por quê?

    R. O produto 111, ele apresentou mais análises positivas, além disso, o produto 222 apresentou críticas quanto a embalagem e logística, o que é crucial para presentear, o produto 222 apresentou uma queda significativa nas compras no mês 04, o que pode ter sido justificado pelas críticas de logística e embalagem, o que faz o produto 111 tornar-se mais relevante, mesmo com menor número de avaliações

3) Faça uma análise exploratória dos dados. Existem outros insights que você acha
relevante de serem compartilhados?

    R. Foi feita uma análise da distribuição por estado de cada um dos produtos e os seus ‘clusters’, revelando uma alta concentração em SP, MG e RJ, além disso, a maioria das avaliações é realizada via ‘app’, a composição das palavras chave, no geral, revelam um sentimento mais positivo que negativo em relação aos produtos